## *Bulk* Geocoding using the Open Street Map API

Building off the last geocoding exercise, here we perform bactch geocoding. We'll read in a set of addresses stored in an Excel file and geocode them using the OSM geocoding API. The addresses are stored in the `ChileCare_Facilities.csv` file in the `Data` folder of this workspace, and we'll save the output in that folder as well.

In [1]:
#Import the requests module
import requests
import pandas as pd
import numpy as np

Create a function that takes the 4 inputs (Street, City, State, Zip) and sends that to the Nomanatim service to be converted into coordinates. If Nominatim can't interpet the address, we'll return a null object.

In [2]:
#Create a function that will invoke the OSM API to return coordinates for an address
def getCoord(x):#street,city,state,zipcode):
    #Form the address
    address = "{0} {1} {2} {3}".format(x[0],x[1],x[2],x[3])
    #Form the request
    osmURL = 'http://nominatim.openstreetmap.org/search'
    params = {'format':'json','q':address} 
    #Try the request
    try:
        response = requests.get(osmURL, params)
        #Read in the response as a JSON encoded object
        jsonObj = response.json()
        d = jsonObj[0]
        lat = float(d['lat'])
        lng = float(d['lon'])
        #Return the coordinate pair for the address
        return(lat, lng)
    #Return a null value if no point is found
    except:
        return None

Read in the data with addresses

In [3]:
#Pull in the excel spreadsheet as a pandas dataframe
df = pd.read_csv('./data/Childcare_Facilities.csv',index_col='Facility ID')
df.columns

Index(['County Name Of Facility Location', 'Facility Name', 'Facility Type',
       'Phone Number', 'Permit Type', 'Email Address',
       'Location Address Line 1', 'Location City', 'Location State',
       'Location Zip Code', 'Full Address ', 'Mailing Address Line 1',
       'Mailing City', 'Mailing State', 'Mailing Zip Code'],
      dtype='object')

In [4]:
#Subset to the columns we need
dfSubset = df[['Location Address Line 1','Location City','Location State','Location Zip Code']].copy(deep=True)

#Drop rows with no address values
dfSubset.dropna(inplace=True,axis='rows')
dfSubset.head()

,Location Address Line 1,Location City,Location State,Location Zip Code
Facility ID,,,,
01000044,3750 ALAMANCE ROAD,BURLINGTON,NC,27215
01000109,721 N FISHER ST,BURLINGTON,NC,27217
01000136,810 N MAIN ST,GRAHAM,NC,27253
01000203,823 LAKESIDE AVE,BURLINGTON,NC,27217
01000214,620 RAY STREET,GRAHAM,NC,27253


In [5]:
#Subset to the columns we need and drop rows with any null values
dfSubset = (df
            .loc[:,['Location Address Line 1','Location City','Location State','Location Zip Code']]
            .dropna(axis='rows')
           )

dfSubset.head()

,Location Address Line 1,Location City,Location State,Location Zip Code
Facility ID,,,,
01000044,3750 ALAMANCE ROAD,BURLINGTON,NC,27215
01000109,721 N FISHER ST,BURLINGTON,NC,27217
01000136,810 N MAIN ST,GRAHAM,NC,27253
01000203,823 LAKESIDE AVE,BURLINGTON,NC,27217
01000214,620 RAY STREET,GRAHAM,NC,27253


Now we'll map our function to the subset dataframes, using only the first 10 records (so we don't overtax the servers just for a demonstration...).

In [6]:
#Iterate through rows and compute the address
dfSubset['Coords'] = dfSubset.iloc[:10].apply(getCoord,axis=1)

In [7]:
#Have a look
dfSubset.head(10)

,Location Address Line 1,Location City,Location State,Location Zip Code,Coords
Facility ID,,,,,
01000044,3750 ALAMANCE ROAD,BURLINGTON,NC,27215,"(36.04282848336784, -79.48649292086625)"
01000109,721 N FISHER ST,BURLINGTON,NC,27217,"(36.10289, -79.429184)"
01000136,810 N MAIN ST,GRAHAM,NC,27253,"(36.079225, -79.405644)"
01000203,823 LAKESIDE AVE,BURLINGTON,NC,27217,"(36.11234419159867, -79.44220962289695)"
01000214,620 RAY STREET,GRAHAM,NC,27253,"(36.06138043894745, -79.39425879746685)"
01000216,808 OAKLEY ST,GRAHAM,NC,27253,"(36.079240495040736, -79.4103780916162)"
01000231,639 SOUTH 8TH ST,MEBANE,NC,27302,"(36.085787182427424, -79.26843035568317)"
01000244,827 S SELLERS MILL RD,BURLINGTON,NC,27217,None
01000253,3216 NC HWY 54 EAST,GRAHAM,NC,27253,None


We see that not all rows got a valid address Why? Perhaps it has something to do with the street addresses having interesting abbreviations ("S" for "South", "NC Hwy"...)